# Conformation Layer

@Author: 吴炜坤

@email：weikun.wu@xtalpi.com weikunwu@163.com


在前一个章节中，我们已经介绍了在Rosetta中，Residue对象是描述蛋白质的基本单元，许多独立的Residue被用于描述蛋白质的几何结构和高级构象。
这些细微的构象变化的度量就是由**原子间的键长、键角，二面角等一系列的具体参数构成。**在PyRosetta中, Pose中这些几何构象的参数由Conformation对象负责记录。

### 一、蛋白质的几何构象
蛋白质是由多个氨基酸通过脱水缩合的方式形成肽键共价连接。因此对于天然氨基酸而言。骨架最重要的两个二面角就是phi和psi角，而omega由于肽键平面一般处于0或180°附近。对于不同的氨基酸侧链，每个Residue含有若干个chi角。这些二面角组成了Rosetta对构象采样的基本几何参数。

<center><img src="./img/phipsiomega.png" width = "600" height = "200" align=center /></center>

In [1]:
# 读取多肽的PDB结构
from pyrosetta import init, pose_from_pdb
init()
pose = pose_from_pdb('./data/4jfx_peptide.pdb')

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release 2021.26+release.b308454c455dd04f6824cc8b23e54bbb9be2cdd7 2021-07-02T13:01:54] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release r288 2021.26+release.b308454c455 b308454c455dd04f6824cc8b23e54bbb9be2cdd7 http://www.pyrosetta.org 2021-07-02T13:01:54
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-1636304719 seed_offset=0 real_seed=-1636304719 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-1636304719 RG_type=mt19937
core.chemical.GlobalRe

#### 1.1 基本二面角几何参数
从pyrosetta中获取这些基本几何参数的方式非常简单:

In [2]:
# 获取第3号氨基酸的骨架二面角:
phi = pose.phi(3)
psi = pose.psi(3)
omega = pose.psi(3)
print(phi, psi, omega)

-148.8025511852094 157.0624491251048 157.0624491251048


也可以直接获取侧链的二面角参数: 

pose.chi($\chi$角编号, Residue的pose编号)即可获取。

In [3]:
# 获取第三号残基的chi角信息: pose.chi(chi_id:int, residue_id:int)
chi_num = len(pose.residue(3).chi_atoms())
print(chi_num)

3


残基中共有3个$\chi$二面角。

In [4]:
# 打印每个chi二面角的
for chi_id in range(1, chi_num+1):
    chi_angle = pose.chi(chi_id, 3)
    print(chi_angle)

70.4233120899352
81.97278776078356
-2.023205011879734e-14


#### 1.2 调整二面角几何参数
Pose对象中内置的几个函数非常方便地可以用于调整几何构象: set_phi, set_psi, set_omega, set_chi。

In [5]:
# 调整骨架二面角
pose.set_phi(seqpos=3, setting=-150)
pose.set_phi(seqpos=3, setting=170)
pose.dump_pdb('./data/4jfx_peptide_conf0.pdb')

True

调整构象后的多肽构象直观感受:
<center><img src="./img/conf.png" width = "500" height = "200" align=center /></center>

In [6]:
# 调整3号氨基酸的侧链chi1角的角度;
pose.set_chi(chino=1, seqpos=3, setting=60)
pose.dump_pdb('./data/4jfx_peptide_chi_conf0.pdb')
pose.set_chi(chino=1, seqpos=3, setting=-60)
pose.dump_pdb('./data/4jfx_peptide_chi_conf1.pdb')
pose.set_chi(chino=1, seqpos=3, setting=180)
pose.dump_pdb('./data/4jfx_peptide_chi_conf2.pdb')

True

<center><img src="./img/chi_conf.png" width = "500" height = "200" align=center /></center>

### 二、化学键几何参数
除了对构象变化依赖影响最大的二面角参数，局部的化学键键长和键角信息也储存在Conformation对象中。
为了定位原子的信息，首先需要构建atom identifier对象，相当于创建一个ID卡，让Rosetta知道我们指定的原子是位于哪个氨基酸中的。通过AtomID，提供残基号，原子号，就可以创建atom identifier对象

In [7]:
# 获取原子间的键长、键角信息前需要构建atom identifier objects
from pyrosetta.rosetta.core.id import AtomID
atom1 = AtomID(atomno_in=1, rsd_in=3)  # 3号残基的第一个原子
atom2 = AtomID(atomno_in=2, rsd_in=3)  # 3号残基的第二个原子
atom3 = AtomID(atomno_in=3, rsd_in=3)  # 3号残基的第三个原子
atom4 = AtomID(atomno_in=4, rsd_in=3)  # 3号残基的第四个原子
print(atom1)
print(atom2)
print(atom3)
print(atom4)

 atomno= 1 rsd= 3 
 atomno= 2 rsd= 3 
 atomno= 3 rsd= 3 
 atomno= 4 rsd= 3 


知道原子的ID后，就可以通过conformation对象来获取键长、键角等数据了。但一般这些参数在Rosetta中键长和键角都设定为理想值，可以极大减少蛋白质构象的采样自由度空间。但注意的是，获取的键长键角必须有是“物理连接的”。

In [8]:
# 通过conformation层获取键长数据
bond_length = pose.conformation().bond_length(atom1, atom2)

# 通过conformation层获取键角数据(弧度)
bond_angle = pose.conformation().bond_angle(atom1, atom2, atom3)

print(f'键长:{bond_length}, 键角:{bond_angle}')

键长:1.4632750937537378, 键角:1.9840915800459624


同样原子间的键长和键角也是可以被调整的:

In [9]:
# 设置新的值:
pose.conformation().set_bond_length(atom1, atom2, setting=1.5)
pose.conformation().set_bond_angle(atom1, atom2, atom3, setting=3.4) # 角度，不是弧度

# 查看新的值设定情况:
new_bond_length = pose.conformation().bond_length(atom1, atom2)
new_bond_angle = pose.conformation().bond_angle(atom1, atom2, atom3)
print(new_bond_length, new_bond_angle)

1.5 3.4


### 三、原子xyz坐标
原子坐标的修改需要获取residue对象，并获取原子ID(atom identifier objects)。通过pose.set_xyz函数设定新的xyz坐标, 但用户一般不需要”显式“地修改原子坐标, 除非你明白这样操作的意义。此处我们沿着3个坐标轴平移所有原子3个埃的距离。

In [10]:
# 原子坐标的修改（一般不需要这样操作）
from pyrosetta.rosetta.numeric import xyzVector_double_t

# 对所有氨基酸的所有原子的x坐标乘上一个负号:
for residue_id in range(1, pose.total_residue()+1):
    residue = pose.residue(residue_id) # 获取residue对象
    for atom_id, atom in enumerate(residue.atoms()):
        x, y, z = atom.xyz()
        
        # 镜像处理xyz坐标:
        trans_xyz = xyzVector_double_t(x+3, y+3, z+3)  # 乘上负号.
        atom_index = AtomID(atom_id+1, residue_id)   # 3号氨基酸的第x个原子的id
        pose.set_xyz(atom_index, trans_xyz) # 设置xyz坐标

In [11]:
pose.dump_pdb('./data/trans.pdb')

True

<center><img src="./img/trans_xyz.png" width = "600" height = "200" align=center /></center>

### 四、 蛋白二级结构信息
Rosetta中的二级结构信息来源于DSSP的计算。通过get_secstruct函数即可获取。

In [12]:
# 通过DSSP获取二级结构信息
from pyrosetta.rosetta.protocols.membrane import get_secstruct
ss = ''.join(get_secstruct(pose))
print(ss)

protocols.DsspMover: {0} LLLLLLLL
LLLLLLLL


### 五、理想化初始拓扑数据
在晶体中，可能会存在一些非理想二面角、键长、键角等。可以通过IdealizeMover进行修复。

In [13]:
from pyrosetta.rosetta.protocols.idealize import IdealizeMover
# idealized
idm = IdealizeMover()
idm.apply(pose)

protocols.idealize.IdealizeMover: {0} total atompairs: 0
protocols.idealize: {0} lastjumpmin: 1
protocols.idealize: {0} premin:  (pos,rmsd,avg-bb,max-bb,avg-chi,max-chi,score)    7 Y    0.432    0.000    0.000    0.000    0.000      13.704
protocols.idealize: {0} postmin: (pos,rmsd,avg-bb,max-bb,avg-chi,max-chi,score)    7 Y    0.424    1.738    2.395    1.691    3.597      12.764
protocols.idealize: {0} 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 pro_close                    0.500       0.000       0.000
 dslf_ss_dst                  0.500       0.000       0.000
 dslf_cs_ang                  2.000       0.000       0.000
 coordinate_constraint        0.010    1276.364      12.764
---------------------------------------------------
 Total weighted score:                       12.764
protocols.idealize: {0} premin:  (pos,rmsd,avg-bb,max-bb,avg-chi

理想化之后，的All-Atom RMSD发生了轻微的变化。

### 习题与思考
1. 尝试下载PDB数据库中的一些数据，统计分析它们的omega角，看看其分布的范围区间？
2. 尝试统计每种氨基酸的Phi/Psi角的分布，它们是否有些什么差异？
3. 不同的二面角它们产生分布差异的原因是什么？